In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from pygam import LinearGAM, s
from sklearn.metrics import r2_score
import numpy as np
from scipy.stats import sem
import math


In [ ]:
from scipy.stats import pearsonr

def calculate_r_squared(y_true, y_pred):
    
    correlation_matrix = np.corrcoef(y_true, y_pred)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2
    return r_squared

def permutation_test_for_p_value(X, y, params, num_permutations=5000):
    actual_r_squared = calculate_r_squared(y, sine_function(X, *params))
    permuted_r_squared = []
    for _ in range(num_permutations):
        y_permuted = np.random.permutation(y)
        permuted_params = fit_sine_model(X, y_permuted)
        permuted_r_squared.append(calculate_r_squared(y_permuted, sine_function(X, *permuted_params)))
    p_value = np.sum(np.array(permuted_r_squared) >= actual_r_squared) / num_permutations
    return p_value


In [ ]:


def calculate_sem_excluding_nan(data):
    """
    Calculate the Standard Error of the Mean (SEM), excluding NaN values.

    Args:
    - data: NumPy array or pandas DataFrame with shape (n_samples, n_features).

    Returns:
    - sem_values: NumPy array containing the SEM values for each column.
    """
    if isinstance(data, pd.DataFrame):
        
        data = data.values
    
 
    non_nan_counts = np.sum(~np.isnan(data), axis=0)
    
  
    std_devs = np.nanstd(data, axis=0, ddof=0)
    
  
    sem_values = std_devs / np.sqrt(non_nan_counts)
    
    return sem_values


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math

def sine_function(x, amplitude, frequency, phase_shift, vertical_shift):
    return amplitude * np.sin(frequency * x + phase_shift) + vertical_shift

def fit_sine_model(X, y):
    # Initial guess for the parameters
    initial_guess = [np.std(y), 2 * np.pi / 12, 0, np.mean(y)]
    
    # Curve fitting
    params, params_covariance = curve_fit(sine_function, X, y, p0=initial_guess, maxfev=5000)

    return params

def plot_sine_model(data, num, save_path=None):
    np.random.seed(100) 
    
    month_names = {
        1: 'January', 2: 'February', 3: 'March', 4: 'April', 
        5: 'May', 6: 'June', 7: 'July', 8: 'August', 
        9: 'September', 10: 'October', 11: 'November', 12: 'December'
    }
    
    means = np.nanmean(data, axis=0)  

    means_for_plotting = np.where(np.isnan(means), 0, means)
    
    std_devs = calculate_sem_excluding_nan(data)  
    
    std_devs_for_plotting = np.where(np.isnan(std_devs), 0, std_devs)
    
    months = np.arange(1, 13) 
    
    valid_indices = ~np.isnan(means)
    valid_months = months[valid_indices]
    valid_means = means[valid_indices]
    

    params = fit_sine_model(valid_months, valid_means)
    fitted_y = sine_function(valid_months, *params)
    
  
    r_squared = calculate_r_squared(valid_means, fitted_y)
    
   
    p_value = permutation_test_for_p_value(valid_months, valid_means, params, num_permutations=5000)

    
   
    proposed_upper_ylim = max(means_for_plotting) + max(std_devs_for_plotting) * 1.5

   
    min_upper_ylim = 20


    actual_upper_ylim = max(proposed_upper_ylim, min_upper_ylim)

   
    actual_upper_ylim = math.ceil(actual_upper_ylim / 5) * 5
    
   

    means_for_plotting = np.where(np.isnan(means), 0, means)

    std_devs = calculate_sem_excluding_nan(data)  
    
    std_devs_for_plotting = np.where(np.isnan(std_devs), 0, std_devs) 
    
    
    
    plt.style.use('seaborn-v0_8-colorblind')
    fig, ax = plt.subplots(figsize=(10, 6))
    
   
    ax.errorbar(months, means_for_plotting, yerr=std_devs, fmt='o', color='darkblue', ecolor='lightblue', elinewidth=2, capsize=5)

    ax.plot(months, means, 'o', color='darkblue', label='Data Points')
    fitted_x = np.linspace(1, 12, 100)
    fitted_y_full_range = sine_function(fitted_x, *params)
    ax.plot(fitted_x, fitted_y_full_range, label='Fitted Sine Curve', color='red')
   
    
    ax.text(0.05, 0.95, f'Fitness Score (R-squared): {r_squared:.3f}\nP-value: {p_value:.3f}',
    horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, 
     fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
    
    
    
    ax.yaxis.set_major_locator(MultipleLocator(5))

    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.set_xlabel('Litter in', fontsize=12)
    ax.set_ylabel('% offspring in month',fontsize=12)
    ax.set_title(month_names[num], fontsize=14)
    ax.set_xticks(months)
   
    ax.set_xlim(0.5, 12.5)

    ax.set_ylim(0, actual_upper_ylim)
    
    if save_path:
        plt.savefig(save_path)
    
    plt.show()

    return r_squared, p_value


In [ ]:

np.random.seed(100) 


def process_data(data, species_name, data_type='Dam'):
    np.random.seed(100) 
    all_scores = []
    month_scores = []
    
    
    birth_month_columns = df.filter(like='BirthMonth').columns
    
    parent_column = birth_month_columns[0]
    
    for month in range(1, 13):
        values=[]
        
        values = data[data[parent_column] == month].iloc[:, 1:]  
        
        values = values.dropna(how='all')
    
        filename = f'./SINE/{species_name} -{data_type} born in {month} (SINE).png' 

        r_squared, p_value = plot_sine_model(values, month, save_path=filename) 
         
   
        month_scores.append({'Month': month, 'Fitness Score': r_squared, 'P-value': p_value})
        
    
        
    
    all_scores.extend(month_scores)
        
  
    scores_df = pd.DataFrame(all_scores)
    scores_df['Species'] = species_name  
   
    return scores_df



In [ ]:


species_list = ['EP', 'IS','SM2', 'BW', 'LL', 'PO'] 

Dam_Sire =['Dam','Sire']


for parent in Dam_Sire:
    np.random.seed(100) 
    combined_scores_df = pd.DataFrame()
    for species in species_list:
        
        file_path= f'./percentage offspring/{species}-{parent}-percentage of offspring by parent birthmonth.xlsx'
     
        df= pd.read_excel(file_path)
        df = df.iloc[:,:-1]
        
                
      
        species_scores_df = process_data(df, species, parent)
        
   
        combined_scores_df = pd.concat([combined_scores_df, species_scores_df], ignore_index=True)

        combined_scores_df.to_excel(f'./SINE/{parent} -SINE- R-score and p-value -all species.xlsx', index=False, engine='openpyxl')

